In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
run_name = "dummy-aokLsWSiStLL2Gdyd3zvLs"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        high_contradiction = get_idx_with_top_column_values('contradiction_scores',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i+1, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['label'])[0] )
    pp_all = list(df1['pp'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp","pp_truelabel_probs","vm_scores","sts_scores","pp_letter_diff", "contradiction_scores", "reward","label_flip", 'pp_logp',
       'ref_logp', 'kl_div', 'reward_penalty', 'loss']
    #g_fields = ["pp","vm_scores"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats_training_step(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

def show_random_examples_for_eval_set(df, n): 
    random_idx = df.idx.drop_duplicates().sample(n).tolist()
    cols = ['epoch','pp', 'is_adv_example', 'is_valid_pp', 'label_flip', 
            'reward', 'sts_scores', 'contradiction_scores', 'pp_letter_diff', 'edit_distance_token_level']
    metric_cols = [o for o in cols if o not in ['pp', 'epoch']]
    agg_d = {k:np.mean for k in metric_cols}
    for i, idx in enumerate(random_idx): 
        print("\n###############\n")
        print("random", i+1, "\n")
        df1 = df.query("idx==@idx")
        print("Original:", pd.unique(df1['orig'])[0])
        print("Original label", pd.unique(df1['label'])[0] )
        df2 = df1[cols]
        print("Paraphrase-level view")
        df3 = df2.groupby('pp').agg({'epoch': lambda x: list(x), **agg_d})
        df3.sort_values(by ='epoch', key=lambda col: col.map(lambda x: np.max(x)))
        display_all(df3)
        print("Epoch-level view")
        df4 = df2.groupby('epoch').agg(agg_d)
        df4.columns = [o + "_avg" for o in df4.columns]
        display_all(df4)
    #     print("Whole table")
    #     display_all(df2)
    return 

In [ ]:
split = 'training_step'
n=3

df_d = get_training_dfs(path_run, postprocessed=True)
df = df_d[split]
if split == "training_step":
    idx_d = get_interesting_idx(df, n)
    print_interesting_text_stats_training_step(df, n)
else: 
    show_random_examples_for_eval_set(df, n)

No label flips detected
No label flips detected

###############

random 1 

Original: I hate this apple
Original label 0
Unique paraphrases: 6
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp,pp_truelabel_probs,vm_scores,sts_scores,pp_letter_diff,contradiction_scores,reward,label_flip,pp_logp,ref_logp,kl_div,reward_penalty,loss,
Tell me your opinion I hate that ome a ne a n he hams?,0.81883,0.01924,0.35473,-37,0.271065,0.00000,0,-2.875007,-3.289462,0.414454,-0.082891,0.0,[1]
I hate this apple.,0.88375,-0.04569,0.94178,-1,0.004530,0.00000,0,-0.282578,-0.155735,0.000000,-0.000000,0.0,[2]
"I hate apple.""",0.82966,0.00840,0.86723,3,0.004348,0.10086,0,-1.755166,-2.869405,1.114239,-0.222848,0.0,[3]
If this apple isn't on the map. Oh me narciss!,0.73735,0.10071,0.58053,-29,0.180517,0.00000,0,-2.641130,-3.438840,0.797709,-0.159542,0.0,[4]
This apple - I hate it so much.,0.86253,-0.02446,0.92733,-14,0.005069,0.00000,0,-1.263933,-1.233151,0.000000,-0.000000,0.0,[5]
I'm against this apple.,0.88164,-0.04357,0.78075,-6,0.003788,0.00000,0,-1.000181,-1.219952,0.219772,-0.043954,0.0,[6]


Best Paraphrase


,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,reward,pp_logp,ref_logp,kl_div,reward_penalty,reward_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
20,3,3,I hate this apple,"I hate apple.""",0.838063,0.82966,0.829659,0,0,0,0.0084,0.86723,3,0.823529,0.00434767,0.10086,-1.75517,-2.8694,1.11424,-0.222848,0,0,1,5,1,1.08265,0.270663,0,16,2,7,2,0.149311,0.26786,0.0366611,0.0101228,0.0145464,0.000159661,0.0109159,0.00858065,0.00191549,0.00498117,0.222004,0.000228404,0.171866,0.0168127,0.072132,0.0325891,6,5,1,4,1,0,0,14,3,1,2,0,10,1,0,-2,0,4,[2],[this],"[4, 5]","[.""]","[0, 1, 3]","[I hate, apple]",1,1,2,1,False,False,False,3



###############

random 2 

Original: I like this movie
Original label 1
Unique paraphrases: 6
How the paraphrases change:


...

,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,reward,pp_logp,ref_logp,kl_div,reward_penalty,reward_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
20,3,3,I hate this apple,"I hate apple.""",0.838063,0.82966,0.829659,0,0,0,0.0084,0.86723,3,0.823529,0.00434767,0.10086,-1.75517,-2.8694,1.11424,-0.222848,0,0,1,5,1,1.08265,0.270663,0,16,2,7,2,0.149311,0.26786,0.0366611,0.0101228,0.0145464,0.000159661,0.0109159,0.00858065,0.00191549,0.00498117,0.222004,0.000228404,0.171866,0.0168127,0.072132,0.0325891,6,5,1,4,1,0,0,14,3,1,2,0,10,1,0,-2,0,4,[2],[this],"[4, 5]","[.""]","[0, 1, 3]","[I hate, apple]",1,1,2,1,False,False,False,3



###############

high_contradiction 3 

Original: I like this movie
Original label 1
Unique paraphrases: 6
How the paraphrases change:


,,,,,,,,,,,,,epoch
pp,pp_truelabel_probs,vm_scores,sts_scores,pp_letter_diff,contradiction_scores,reward,label_flip,pp_logp,ref_logp,kl_div,reward_penalty,loss,
I like this movie,0.70617,0.00000,1.00000,0,0.002428,0.0,0,-0.611121,-0.743659,0.132539,-0.026508,0.0,[1]
I like that movie.,0.77629,-0.07013,0.87722,-1,0.127013,0.0,0,-0.884103,-1.140530,0.256427,-0.051285,0.0,[2]
I love this movie.,0.92497,-0.21880,0.88994,-1,0.001813,0.0,0,-0.633803,-0.830810,0.197007,-0.039401,0.0,[3]
I like this movie!,0.81051,-0.10434,0.94670,-1,0.002176,0.0,0,-0.790599,-0.923505,0.132906,-0.026581,0.0,[4]
I like this movie :,0.71569,-0.00953,0.95414,-2,0.002361,0.0,0,-0.782485,-0.575688,0.000000,-0.000000,0.0,[5]
I like this movie.,0.82852,-0.12236,0.96749,-1,0.002737,0.0,0,-0.454331,-0.264591,0.000000,-0.000000,0.0,[6]


Best Paraphrase


,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,reward,pp_logp,ref_logp,kl_div,reward_penalty,reward_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
0,0,1,I like this movie,I like this movie,0.706167,0.70617,0.706167,1,1,0,0,1,0,1,0.00242789,0,-0.611121,-0.743659,0.132539,-0.0265077,0,0,0,0,0,0,0,0,16,2,10,2,0.236227,0.366817,0.0380617,0.00956986,0.0162048,0.000141597,0.0111624,0.0103769,0.00284763,0.00523393,0.317811,0.000138477,0.244319,0.016125,3.93819e-05,5.19492e-06,6,5,1,4,1,0,0,14,4,1,0,0,14,0,0,0,0,0,[],[],[],[],"[0, 1, 2, 3]",[I like this movie],0,0,0,0,False,False,False,0


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

not                   7
this                  5
love                  3
film                  2
movie                 2
apple                 2
like                  1
hate                  1
I do not like this    1
this apple            1
do                    1
I do not like         1
I love                1
dtype: int64

#### INSERTIONS ####

.                                    21
n't                                   7
like                                  2
that                                  2
film.                                 2
- I like it                           1
thyme                                 1
really                                1
Not so much                           1
'.                                    1
:                                     1
''.                                   1
love                                  1
ie                                    1
cherry.                               1
enjoy                          

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

ValueError: a must be greater than 0 unless no samples are taken